In [1]:
from gerrychain import Graph, Partition, Election, MarkovChain, proposals, updaters, constraints, accept, GeographicPartition
from gerrychain.updaters import Tally, cut_edges
from networkx import is_connected, connected_components
import geopandas as gdp
import pandas as pd
import os

In [9]:
default_path = os.path.join(os.getcwd(), "..", "shapefiles", "processed")
path = os.path.join(default_path, "florida", "precinct_level_w_election.shp")
# new_path = os.path.join(default_path, "florida", "precinct_level_w_election_and_connected.shp")
json_path = os.path.join(default_path, "florida", "precinct_level_w_election_and_connected.json")
graph = Graph.from_json(json_path)

In [11]:
FL_election = Election("SEN", {"Democratic": "D_VOTES", "Republican": "R_VOTES"})

In [14]:
FL_init_partition = GeographicPartition(
    graph, 
    assignment="CONG_DIST", 
    updaters={
        "population": Tally("TOT_POP", alias="population"),
        "white": Tally("WHITE_POP", alias="white"),
        "black": Tally("BLK_POP", alias="black"),
        "asian": Tally("ASN_POP", alias="asian"),
        "hisp": Tally("HSP_POP", alias="hisp"),
        "cut_edges": cut_edges,
        "SEN": FL_election})
FL_init_partition.plot()
FL_init_partition["other"]

AttributeError: 'Graph' object has no attribute 'geometry'

In [10]:
is_connected(graph)

True

In [5]:
components = list(connected_components(graph))
[len(c) for c in components]

[4790, 2, 1, 3, 3, 1, 3, 1, 2]

In [6]:
biggest_component_size = max(len(c) for c in components)
problem_components = [c for c in components if len(c) != biggest_component_size]
for component in problem_components:
    for node in component:
        graph.remove_node(node)
[len(c) for c in components]

[4790, 2, 1, 3, 3, 1, 3, 1, 2]

In [7]:
is_connected(graph)

True

In [8]:
graph.to_json(json_path)